In [2]:
import sys
sys.path.append('/home/djuhera/es-sbm-master')
import torch
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from typing import Tuple, List, Dict, Any
import vis
import degcor
import overlapping
import bernoulli
import time
from contextlib import contextmanager

GPU used:  True


In [3]:
DEV = 'cpu'

# Suppress Console Output
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout
            

def make_utilities(population_size: int) -> torch.Tensor:
    """
    Make utility values to weight samples with.

    Args:
        population_size:

    Returns:
        utility_values: Shape (S,).
    """
    utility_values = np.log(population_size / 2. + 1) - \
                     np.log(np.arange(1, population_size + 1))
    utility_values[utility_values < 0] = 0.
    utility_values /= np.sum(utility_values)
    utility_values -= 1. / population_size
    utility_values = utility_values.astype(np.float32)
    return torch.tensor(utility_values)


def scale_noise(noise: torch.Tensor, fitness: torch.Tensor, log_probs: torch.Tensor) -> torch.Tensor:
    """
    Scale noise with the fitness values. Fitness values are assigned to noise
    vectors based on the `log_probs` vector. The better the `log_probs` vector,
    the higher the fitness and thus the more the gradient should point into
    that direction.

    Args:
        noise: Noise used to perturb the parameters (S, |V|, k).
        fitness: Vector of scalars, first element `fitness[0]` is the fitness corresponding
            to the best sample, the last element `fitness[-1] is the fitness
            for the worst sample. Has shape (S).
        log_probs: Log prob resulting from the sampled placements. Has shape (S,).

    Returns:
        scaled_noise: Noise scaled with fitness values. Has shape (S, |V|, d).
    """
    sorted_values, indices = torch.sort(-1. * log_probs)
    scaled_noise = fitness.unsqueeze(1).unsqueeze(2) * noise[indices, :, :]
    return scaled_noise


def apply_gradient(params: torch.Tensor, grads: torch.Tensor, lr: float) -> torch.Tensor:
    """
    Apply the gradient to the parameters.

    Args:
        params: Tensor with parameters of shape (|V|, d).
        grads: Tensor with gradients of shape (|V|, d).

    Returns:
        params_prime: Tensor of shape (|V|, d).
    """
    params_prime = params + lr * grads
    return params_prime


def calculate_gradient(scaled_noise: torch.Tensor) -> torch.Tensor:
    """
    Calcualte the gradient from the scaled noise.

    Args:
        scaled_noise: Tensor of shape (S, |V|, d).

    Returns:
        grad: Tensor of shape (|V|, d).
    """
    grad = torch.sum(scaled_noise, 0)
    return grad


def sample_placements(mutated_params: torch.Tensor) -> torch.Tensor:
    """
    Sample indices from mutated paramters.

    Args:
        mutated_params: Shape (S, |V|, k)

    Returns:
        memberships: Shape (S, |V|).
    """
    # cumsum = torch.cumsum(torch.softmax(mutated_params, dim=-1), dim=-1)
    # noise = torch.rand(mutated_params.shape[0], mutated_params.shape[1], 1, requires_grad=False)
    # indices = torch.argmin((cumsum < noise).int(), dim=-1)
    indices = torch.distributions.Categorical(logits=mutated_params).sample()
    return indices


def sample_propensities(mutated_params: torch.Tensor) -> torch.Tensor:
    """
    Create propensities for parameters, i.e., soft assignment.

    Args:
        mutated_params: Shape (S, |V|, k)

    Returns:
        memberships: Shape (S, |V|, k).
    """
    return torch.softmax(mutated_params, -1)


def mutate_params(params: torch.Tensor, num_samples: int, scale: float) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Create `num_samples` mutation of parameters.

    Args:
        params: Shape (|V|, k)

    Returns:
        mutated_params: Shape (S, |V|, k)
    """
    noise = torch.randn([num_samples, params.shape[0], params.shape[1]], requires_grad=False)
    mutated_params = torch.unsqueeze(params, 0) + noise
    return noise, mutated_params


def calc_scale_decrease(num_iter: int, fraction: float, target_scale: float) -> float:
    stop = num_iter * fraction
    m = (target_scale - 1.) / stop
    return m


def learned_embeddings(constants: Dict[str, torch.Tensor], population_size: int, k: int,
                       num_nodes: int, driver: callable, make_membership: callable,
                       lr: float, num_iter=10000, params: torch.Tensor=None,
                       patience: int=None) -> Dict[str, torch.Tensor]:
    """

    Args:
        constants: Dictionary containing constant arguments to driver functions.
            For example, edges, in_degrees, out_degrees. Check the signature call
            of a specific driver function to determine the fields of this dict.
        population_size: Number of samples for ES.
        k: Number of groups.
        num_nodes: Number of nodes in graph.
        driver: Callable that implements the calculation of the log-prob and
            estimates parameters of a model.
        lr: Learning rate that should be used.
        num_iter: Number of iterations that should be executed.
        params: Initial guess of parameters, if None start from uniform distribution
            over groups.
        patience: Extension of learning after new best log prob has been found.

    Returns:
        Dict
    """
    if params is None:
        # Start from a uniform distribution over group memberships.
        params = torch.ones(num_nodes, k, requires_grad=False, device=DEV)
    fitness = make_utilities(population_size)

    target_scale = 1e-3
    fraction = 0.8
    ascend = calc_scale_decrease(num_iter, fraction, target_scale)
    all_losses = []
    best_loss = 1e9
    best_params = None
    best_eta = None
    best_assignment = None
    patience = int(num_iter * 0.1) if patience is None else patience
    iter = 0
    while iter < num_iter:
        scale = float(np.clip(iter * ascend + 1, target_scale, 1.))
        # scale = 1.
        iter += 1
        noise, mutations = mutate_params(params, population_size, scale)
        assignments = make_membership(mutations)
        log_probs, estimates = driver(assignments, k, **constants)
        all_losses.append(log_probs.cpu().detach().numpy())

        bl = torch.min(-1. * log_probs)
        if bl <= best_loss:
            best_loss = bl.cpu().detach()
            idx = torch.argmax(log_probs)
            best_params = mutations[idx, :, :]
            best_eta = estimates['etas'][idx]
            best_assignment = assignments[idx]
            if iter + patience > num_iter and bl < best_loss:
                num_iter = iter + patience
        if iter % 10 == 0:
            print(iter, scale,  best_loss, torch.mean(log_probs))

        scaled_noise = scale_noise(noise, fitness, log_probs)
        grads = calculate_gradient(scaled_noise)
        params = apply_gradient(params, grads, lr)

    return {
        "best_eta": best_eta.cpu(),
        "best_assignment": best_assignment.cpu(),
        "best_params": best_params.cpu(),
        "best_log_prob": -1. * bl,
        "params": params.cpu()
    }


In [4]:
g = nx.karate_club_graph()
edges = list(g.to_directed().edges())
edges = [(u, v, 1) for u, v in edges]
edges = torch.tensor(edges, requires_grad=False)

in_degrees = torch.tensor([float(g.to_directed().in_degree[v]) for v in g.nodes()])
out_degrees = torch.tensor([float(g.to_directed().out_degree[v]) for v in g.nodes()])

constants = {
    'edges': edges,
    'in_degrees': in_degrees,
    'out_degrees': out_degrees
}
constants = {
    'edges': edges
}

embeddings = learned_embeddings(
    constants=constants,
    population_size=10000,
    k=2,
    num_nodes=g.number_of_nodes(),
    driver=overlapping.driver,#degcor.driver,
    make_membership=sample_propensities,#sample_placements,
    lr=1,
    num_iter=600
)

#print(embeddings)
#print(torch.softmax(embeddings['best_params'], -1))
#print(torch.softmax(embeddings['params'], -1))

members = torch.clamp(torch.softmax(embeddings['best_params'], -1), 0.001, 0.999).numpy()
for i in range(members.shape[0]):
    for j in range(members.shape[1]):
        print('{:.4f}'.format(members[i, j]), end='\t')
    print()

10 0.98126875 tensor(668.8786) tensor(-696.9060)
20 0.96045625 tensor(628.8748) tensor(-648.9282)
30 0.93964375 tensor(619.0401) tensor(-630.2614)
40 0.91883125 tensor(617.7997) tensor(-627.9886)
50 0.89801875 tensor(617.2100) tensor(-627.4258)
60 0.87720625 tensor(617.0930) tensor(-627.1802)
70 0.85639375 tensor(616.8852) tensor(-627.0475)
80 0.8355812499999999 tensor(616.7005) tensor(-626.9092)
90 0.81476875 tensor(616.5712) tensor(-626.8587)
100 0.7939562499999999 tensor(616.5712) tensor(-626.7922)
110 0.77314375 tensor(616.5712) tensor(-626.7832)
120 0.75233125 tensor(616.5635) tensor(-626.6899)
130 0.73151875 tensor(616.5635) tensor(-626.7875)
140 0.71070625 tensor(616.5635) tensor(-626.6567)
150 0.68989375 tensor(616.5411) tensor(-626.7739)
160 0.66908125 tensor(616.4867) tensor(-626.7245)
170 0.64826875 tensor(616.4867) tensor(-626.7720)
180 0.6274562499999999 tensor(616.4545) tensor(-626.7566)
190 0.6066437499999999 tensor(616.4545) tensor(-626.6299)
200 0.58583125 tensor(616.4

In [5]:
# Runtime Comparison: General
def get_mean_runtime(num_iters):
    durs = []
    for i in range(num_iters):
        with suppress_stdout():
            start = time.time()
            embeddings = learned_embeddings(
                constants=constants,
                population_size=4000,
                k=2,
                num_nodes=g.number_of_nodes(),
                driver=overlapping.driver,#degcor.driver,
                make_membership=sample_propensities,#sample_placements,
                lr=1,
                num_iter=600
            )
            end = time.time()

        duration = end - start 
        durs.append(duration)
        print("Finished iteration #", i)
    
    mean_dur = sum(durs)/num_iters
    
    return mean_dur, durs


mean_dur, durs = get_mean_runtime(10)
print("\nMean Duration until Convergence: ", mean_dur)


Finished iteration # 0
Finished iteration # 1
Finished iteration # 2
Finished iteration # 3
Finished iteration # 4
Finished iteration # 5
Finished iteration # 6
Finished iteration # 7
Finished iteration # 8
Finished iteration # 9

Mean Duration until Convergence:  14.813968777656555


In [6]:
# Runtime Comparison: Iterative
def get_mean_runtime(num_iters):
    durs = []
    for i in range(num_iters):
        with suppress_stdout():
            start = time.time()
            embeddings = learned_embeddings(
                constants=constants,
                population_size=4000,
                k=2,
                num_nodes=g.number_of_nodes(),
                driver=overlapping.driver,#degcor.driver,
                make_membership=sample_propensities,#sample_placements,
                lr=1,
                num_iter=1
            )
            end = time.time()

        duration = end - start 
        durs.append(duration)
        print("Finished iteration #", i)
    
    mean_dur = sum(durs)/num_iters
    
    return mean_dur, durs


mean_dur, durs = get_mean_runtime(10)
print("\nMean Duration for one Iteration: ", mean_dur)
print("\nMean Iters/s: ", round(1/mean_dur))

Finished iteration # 0
Finished iteration # 1
Finished iteration # 2
Finished iteration # 3
Finished iteration # 4
Finished iteration # 5
Finished iteration # 6
Finished iteration # 7
Finished iteration # 8
Finished iteration # 9

Mean Duration for one Iteration:  0.028452515602111816

Mean Iters/s:  35
